In [7]:
import pandas as pd

df1 = pd.read_csv('cursoGraduacao.csv', sep=';')

df2 = pd.read_csv('matriculasCursosGraduacao.csv', sep=';')


df2.head(10)

,matricula,curso,modalidade_entrada,ano_ingresso,data_colacao_grau,situacao_matricula,sexo,prazo_conclusao,id_curso,data_registro_diploma
0,201*****,SISTEMAS DE INFORMAÇÃO,ESCOLA PUBLICA - RENDA FAMILIAR INFERIOR - PRE...,2018,NaN,ATIVO,M,20231.0,518224,NaN
1,170*****,ENGENHARIA METALÚRGICA,AMPLA CONCORRENCIA,2017,NaN,ATIVO,M,20272.0,518225,NaN
2,150*****,FÍSICA,ESCOLA PUBLICA - INDEPENDENTE DA RENDA,2015,NaN,CANCELADO,M,NaN,518227,NaN
3,202*****,SISTEMAS PARA INTERNET,ESCOLA PUBLICA - INDEPENDENTE DA RENDA PRETO/P...,2021,NaN,CANCELADO,F,20252.0,527820,NaN
4,112*****,ADMINISTRAÇÃO,AMPLA CONCORRENCIA,2011,NaN,CONCLUÍDO,F,20182.0,528322,NaN
5,202*****,GESTÃO AMBIENTAL,AMPLA CONCORRENCIA,2021,NaN,ATIVO,F,20271.0,10154958,NaN
6,201*****,GESTÃO DE TURISMO,AMPLA CONCORRENCIA,2014,21-05-2021,CONCLUÍDO,F,20181.0,527780,20-07-2021
7,130*****,ENGENHARIA MECATRÔNICA,ESCOLA PUBLICA - INDEPENDENTE DA RENDA,2013,NaN,CANCELADO,M,20222.0,518226,NaN
8,102*****,ENGENHARIA MECATRÔNICA,ESCOLA PUBLICA - INDEPENDENTE DA RENDA,2010,NaN,CANCELADO,M,NaN,518226,NaN
9,110*****,ADMINISTRAÇÃO,AMPLA CONCORRENCIA,2011,NaN,CANCELADO,M,NaN,553290,NaN


Limpeza dos dados

In [8]:
# Remover valores ausentes
df1.isnull().sum()
df2.isnull().sum()

df1.dropna(axis=0, inplace=True)
df2.dropna(axis=0, inplace=True)

df1.fillna(df1.mean(), inplace=True)
df2.fillna(df1.mean(), inplace=True)

df1.head()
df2.head()

,matricula,curso,modalidade_entrada,ano_ingresso,data_colacao_grau,situacao_matricula,sexo,prazo_conclusao,id_curso,data_registro_diploma
6,201*****,GESTÃO DE TURISMO,AMPLA CONCORRENCIA,2014,21-05-2021,CONCLUÍDO,F,20181.0,527780,20-07-2021
67,160*****,ADMINISTRAÇÃO,ESCOLA PUBLICA - RENDA FAMILIAR INFERIOR - PRE...,2016,22-10-2020,CONCLUÍDO,F,20232.0,553290,04-02-2021
80,201*****,MATEMÁTICA,ESCOLA PUBLICA - INDEPENDENTE DA RENDA,2018,19-08-2022,CONCLUÍDO,F,20252.0,459794,29-12-2022
89,160*****,ADMINISTRAÇÃO,ESCOLA PUBLICA - RENDA FAMILIAR INFERIOR - PRE...,2016,22-10-2020,CONCLUÍDO,M,20232.0,553290,04-02-2021
94,122*****,FÍSICA,ESCOLA PUBLICA - INDEPENDENTE DA RENDA,2012,30-03-2023,CONCLUÍDO,M,20211.0,518227,12-05-2023


In [9]:
# Selecionando apenas colunas numéricas
numeric_columns = df1.select_dtypes(include=['number']).columns

# Calculando o IQR apenas para colunas numéricas
Q1 = df1[numeric_columns].quantile(0.25)
Q3 = df1[numeric_columns].quantile(0.75)
IQR = Q3 - Q1

# Filtrando outliers apenas para colunas numéricas
df1_no_outliers = df1[~((df1[numeric_columns] < (Q1 - 1.5 * IQR)) | (df1[numeric_columns] > (Q3 + 1.5 * IQR))).any(axis=1)]



Modelo Star Scheme (Separarando em tabelas de fatos e dimensionais) 
cursoGraduacao.csv

In [11]:


CREATE TABLE fato_cursos (
    id_curso INT PRIMARY KEY,
    id_municipio INT,
    id_curso_info INT,
    id_reconhecimento INT,
    data_inicio_funcionamento DATE,
    portaria_reconhecimento VARCHAR(255),
    -- Adicione outras métricas relevantes
    FOREIGN KEY (id_municipio) REFERENCES dim_municipio(id_municipio),
    FOREIGN KEY (id_curso_info) REFERENCES dim_curso_info(id_curso_info),
    FOREIGN KEY (id_reconhecimento) REFERENCES dim_reconhecimento(id_reconhecimento)
);

CREATE TABLE dim_municipio (
    id_municipio INT PRIMARY KEY,
    municipio VARCHAR(255),
);

CREATE TABLE dim_curso_info (
    id_curso_info INT PRIMARY KEY,
    nome_curso VARCHAR(255),
    modalidade_curso VARCHAR(50),
    grau_curso VARCHAR(50),
    ensino VARCHAR(50),
    eixo_conhecimento VARCHAR(255)
);

CREATE TABLE dim_reconhecimento (
    id_reconhecimento INT PRIMARY KEY,
    portaria_reconhecimento VARCHAR(255),
);



SyntaxError: invalid syntax (3330869212.py, line 1)

Modelo Star Scheme (Separarando em tabelas de fatos e dimensionais) 
matriculasCursosGraduacao.csv

In [ ]:
CREATE TABLE fato_matriculas (
    matricula VARCHAR(255) PRIMARY KEY,
    id_curso INT,
    modalidade_entrada VARCHAR(255),
    ano_ingresso INT,
    data_colacao_grau DATE,
    situacao_matricula VARCHAR(255),
    sexo CHAR(1),
    prazo_conclusao INT,
    data_registro_diploma DATE,
    FOREIGN KEY (id_curso) REFERENCES dim_cursos(id_curso)
);

CREATE TABLE dim_cursos (
    id_curso INT PRIMARY KEY,
    curso VARCHAR(255)
);


Criar um script sql para a criação de um banco de dados matriculas da tabelas já geradais (cursoGraduacao.csv)

In [ ]:
-- Criação do banco de dados
CREATE DATABASE IF NOT EXISTS matriculas_database;
USE matriculas_database;

-- Criação da tabela de cursos
CREATE TABLE dim_cursos (
    id_curso INT PRIMARY KEY,
    nome_curso VARCHAR(255),
    municipio VARCHAR(255),
    unidade VARCHAR(255),
    modalidade_educacao VARCHAR(50),
    modalidade_curso VARCHAR(50),
    turno VARCHAR(50),
    grau_curso VARCHAR(50),
    ensino VARCHAR(50),
    tipo_regime_letivo VARCHAR(50),
    situacao_curso VARCHAR(50),
    eixo_conhecimento VARCHAR(50),
    data_inicio_funcionamento DATE,
    codigo_e_mec INT,
    portaria_reconhecimento VARCHAR(255)
);

-- Criação da tabela de matrículas
CREATE TABLE fat_matriculas (
    matricula INT PRIMARY KEY,
    id_curso INT,
    curso VARCHAR(255),
    modalidade_entrada VARCHAR(255),
    ano_ingresso INT,
    data_colacao_grau DATE,
    situacao_matricula VARCHAR(50),
    sexo CHAR(1),
    prazo_conclusao INT,
    data_registro_diploma DATE,
    FOREIGN KEY (id_curso) REFERENCES dim_cursos(id_curso)
);


Populando a tabela (cursoGraduacao.csv)


In [ ]:

INSERT INTO dim_cursos (id_curso, nome_curso, municipio, unidade, modalidade_educacao, modalidade_curso, turno, grau_curso, ensino, tipo_regime_letivo, situacao_curso, eixo_conhecimento, data_inicio_funcionamento, codigo_e_mec, portaria_reconhecimento)
VALUES
(527785, 'QUÍMICA', 'Barbacena', 'BBC-DIRETORIA DE ENSINO', 'Presencial', 'GRADUACAO', 'Noturno', 'LICENCIATURA', 'Hora/Aula', 'Semestral', 'Em Atividade', 'Ciências Exatas e da Terra', NULL, 1103487, 'Portaria MEC nº 728'),
(518226, 'ENGENHARIA MECATRÔNICA', 'Juiz de Fora', 'JFA-DIRETORIA DE ENSINO', 'Presencial', 'GRADUACAO', 'Integral', 'BACHARELADO', 'Hora/Aula', 'Semestral', 'Em Atividade', 'Engenharias', NULL, 123613, 'Portaria SERES/MEC n.º 110');
--etc


INSERT INTO fat_matriculas (matricula, id_curso, curso, modalidade_entrada, ano_ingresso, data_colacao_grau, situacao_matricula, sexo, prazo_conclusao, data_registro_diploma)
VALUES
(201*****, 527785, 'QUÍMICA', 'ESCOLA PUBLICA - RENDA FAMILIAR INFERIOR - PRETO/PARDO/INDIGENA', 2018, NULL, 'ATIVO', 'M', 20231, NULL),
(170*****, 518226, 'ENGENHARIA MECATRÔNICA', 'AMPLA CONCORRENCIA', 2017, NULL, 'ATIVO', 'M', 20272, NULL);
--etc

Criar um script sql para a criação de um banco de dados matriculas da tabelas já geradais (matriculasCursosGraduacao.csv)

In [ ]:

CREATE DATABASE IF NOT EXISTS matriculas;
USE matriculas;

CREATE TABLE IF NOT EXISTS fato_matriculas (
    matricula VARCHAR(255) PRIMARY KEY,
    id_curso INT,
    modalidade_entrada VARCHAR(255),
    ano_ingresso INT,
    data_colacao_grau DATE,
    situacao_matricula VARCHAR(255),
    sexo CHAR(1),
    prazo_conclusao INT,
    data_registro_diploma DATE,
    FOREIGN KEY (id_curso) REFERENCES dim_cursos(id_curso)
);

CREATE TABLE IF NOT EXISTS dim_cursos (
    id_curso INT PRIMARY KEY,
    curso VARCHAR(255)
);


Populando as tabelas (matriculasCursoGraduacao.csv)

In [ ]:

INSERT INTO dim_cursos (id_curso, curso) VALUES
(01, 'SISTEMAS DE INFORMAÇÃO'),
(02, 'ENGENHARIA METALÚRGICA'),
(03, 'FÍSICA'),
(04, 'SISTEMAS PARA INTERNET'),
(05, 'ADMINISTRAÇÃO'),
(06, 'GESTÃO AMBIENTAL'),
(07, 'GESTÃO DE TURISMO'),
(08, 'AGROECOLOGIA'),
(09, 'ALIMENTOS'),
(10, 'ANÁLISE E DESENVOLVIMENTO DE SISTEMAS'),
(11, 'CIÊNCIA DA COMPUTAÇÃO'),
(12, 'CIÊNCIA E TECNOLOGIA DE LATICÍNIOS'),
(13, 'CIÊNCIA E TECNOLOGIA DE ALIMENTOS'),
(14, 'CIÊNCIAS BIOLÓGICAS'),
(15, 'DESENVOLVIMENTO DE SISTEMAS DISTRIBUÍDOS'),
(16, 'DESIGN DE INTERIORES'),
(17, 'DESIGN DE MODA'),
(18, 'DIREITO'),
(19, 'EDUCAÇÃO FISÍCA'),
(20, 'ENGENHARIA ELÉTRICA'),
(21, 'ENGENHARIA FERROVIÁRIA E METROVIÁRIA'),
(22, 'ENGENHARIA MECATRÔNICA'),
(23, 'GESTÃO DE TECNOLOGIA DE INFORMAÇÃO'),
(24, 'GESTÃO DE RECURSOS HUMANOS'),
(25, 'LATICÍNIOS'),
(26, 'LETRAS - LÍNGUA PORTUGUESA E LÍNGUA ESPANHOLA'),
(27, 'LOGÍSTICA'),
(28, 'MATEMÁTICA'),
(29, 'NUTRIÇÃO'),
(30, 'QUÍMICA'),
(31, 'ZOOTECNIA');



INSERT INTO fato_matriculas (matricula, id_curso, modalidade_entrada, ano_ingresso, data_colacao_grau, situacao_matricula, sexo, prazo_conclusao, data_registro_diploma) VALUES
('201*****', 518224, 'ESCOLA PUBLICA - RENDA FAMILIAR INFERIOR - PRETO/PARDO/INDIGENA', 2018, NULL, 'ATIVO', 'M', 20231, NULL),
('170*****', 518225, 'AMPLA CONCORRENCIA', 2017, NULL, 'ATIVO', 'M', 20272, NULL),
('150*****', 518227, 'ESCOLA PUBLICA - INDEPENDENTE DA RENDA', 2015, NULL, 'CANCELADO', 'M', NULL, NULL),
('202*****', 527820, 'ESCOLA PUBLICA - INDEPENDENTE DA RENDA PRETO/PARDO/INDIGENA', 2021, NULL, 'CANCELADO', 'F', 20252, NULL),
('112*****', 528322, 'AMPLA CONCORRENCIA', 2011, NULL, 'CONCLUÍDO', 'F', 20182, NULL),
('202*****', 10154958, 'AMPLA CONCORRENCIA', 2021, NULL, 'ATIVO', 'F', 20271, NULL),
('201*****', 527780, 'AMPLA CONCORRENCIA', 2014, '2021-07-20', 'CONCLUÍDO', 'F', 20181, '2021-07-20');

